In [1]:
import numpy as np
import cv2 as cv2

In [2]:
from sort import *

In [3]:
video_path = 'traffic-1.mp4'
cap = cv2.VideoCapture(video_path)
output_path = 'output_video.mp4'
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
scale = 0.00392

classes = None

with open('yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [4]:
print(fps,width,height)

30 2400 1080


In [5]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [6]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

In [7]:
def draw_prediction(img, name, class_id, x, y, x_plus_w, y_plus_h):
    
    label = str(classes[class_id]) + name

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 5)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img

In [8]:
tracker = Sort()
frames =[]
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

    blob = cv2.dnn.blobFromImage(frame, scale, (416,416), (0,0,0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    detections = []
    for i in indices:
        b_box =[]
        try:
            box = boxes[i]
        except:
            i = i[0]
            box = boxes[i]
    
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        b_box.extend((x,y,x+w,y+h,confidence))
        detections.append(b_box)
    track_ids = tracker.update(np.array(detections))

    for i in range(len(track_ids.tolist())):
        coords = track_ids.tolist()[i]
        x1,y1,x2,y2 = int(coords[0]), int(coords[1]), int(coords[2]), int(coords[3])
        name_id = int(coords[4])
        name = "ID: {}".format(str(name_id))
        modified_frame = draw_prediction(frame, name, class_ids[i], round(x1), round(y1), round(x2), round(y2))
    frames.append(modified_frame)

In [9]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('yolo_sort.mp4', fourcc, fps, (width, height))
for frame in frames:
    out.write(frame)

out.release()